#### Setting the workingspace

In [1]:
from Functions.sorting_functions import *

The "links_file" contains the json file THAT YOU SHOULD ADJUST.

The "output_file" contains the csv file that will be generated here.

Both are located in the directory JobLinks.

In [2]:
current_working_directory = Path.cwd()

links_file = "links.json"
output_file = "Applications.csv"
jobLinks_directory = "JobLinks"

links_path = current_working_directory / jobLinks_directory / links_file
output_path = current_working_directory / jobLinks_directory / output_file

## Extract & Save applications

#### Retrieving links from json (Make sure you adjust the links.json file)
Extracting data from the json file where you should be adding your links, check readme

In [3]:
with open( Path.expanduser(links_path), 'r' ) as file:
    json_data = json.load(file)

#### Extracting applications from the web
Making one big list containing the web links to each application

In [4]:
json_links_count = 0
applications_per_link = []

for json_link in json_data['links']:       # loops every link in the json file
    url = formatting_url(json_link['url'])    # the url should finish with "page=", if not it will be handeled using the formatting_utl func
    number_of_offers = json_link['number_of_offers']
    page_max = math.ceil(number_of_offers/20)
    json_links_count += 1
    print(f'Extracting applications from link number : {json_links_count}')
    applications_per_page = [get_job_links(url+str(page_n)) for page_n in range(page_max)]   # contains the links of the 20 applications in the page_n 
    flat_list_links = [item for sublist in applications_per_page for item in sublist]   #  flatenning the lists into one list
    applications_per_link.append(flat_list_links)  # append the flattened list to the main list

# The main list then contains a list[per link] of lists[applications] 
final_big_list = [item for sublist in applications_per_link for item in sublist]  # making one big list of links

print(f'\n{len(final_big_list)} applications have been extracted !!')

Extracting applications from link number : 1
Extracting applications from link number : 2
Extracting applications from link number : 3

189 applications have been extracted !!


#### Exporting applications in a csv file
Writing each application on a row in a csv file 

In [5]:
with open(str(output_path) , mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    
    writer.writerow([output_file])
    
    # Write each link as a new row
    for link in final_big_list:
        writer.writerow([link])

#### Results

In [6]:
print(f"{len(final_big_list)} applications have been written to {output_file}")

189 applications have been written to Applications.csv


## Auto Apply

#### Retrieving applications links
Reading the list of applications links generated earlier.


In [7]:
links_list = []

with open(str(output_path), mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader, None)  # Skip the header row if present
    
    # Append each link to the list
    for row in reader:
        if row:  # Check if row is not empty
            links_list.append(row[0])  # Access the first column (link)

print(f"Retrieved {len(links_list)} links from the CSV file:")
links_list[:5]

Retrieved 189 links from the CSV file:


['https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/175200696W?motsCles=matlab&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesCon=&page=0&selectedIndex=0',
 'https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/175301272W?motsCles=matlab&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesCon=&page=0&selectedIndex=1',
 'https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/175311278W?motsCles=matlab&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesCon=&page=0&selectedIndex=2',
 'https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/175192925W?motsCles=matlab&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesCon=&page=0&selectedIndex=3',
 'https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/175219199W?motsCles=matlab&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesCon=&page=

#### Setting the application process
cv path, message, username ...

In [8]:
cv_path = '/home/ocr8n/Documents/CV_Aziz_ISC.pdf'      # path to your cv
message = ''    # The message represents what you want to write at every application, a message to the company along with the cv

Username and password are in an .env file, follow the readme for a secure method or change it manually bellow 

In [9]:
# Load environment variables from .env file
load_dotenv() 
username = os.getenv("APEC_USERNAME") # string containing the username (no spaces)
password = os.getenv("APEC_PASSWORD") # string containing the password (no spaces)

Initializing the count

In [10]:
external_links_list = []
links_counter = 0
submitted_counter = 0
applied_counter = 0
external_links_counter = 0
not_available_counter = 0

#### Automatic applications
Send cv and message to each link

In case you runned the code and applied to 50 from a total of 200 and stopped it, you can adjust manually where you stopped and continue from there instead of restarting from the beggining 

When you force the stopping of the process, run the cell bellow to know where you stopped 

In [11]:
# Adjust if not first cell run 
links_counter = 0
submitted_counter = 0
applied_counter = 0
external_links_counter = 0
not_available_counter = 0

# Dont forget to set the values above to 0 for the first launch

for link in links_list[links_counter:]:
    links_counter += 1
    print('________________________________________________\n')
    print(f'Link number : {links_counter}\n')
    print('\n First step, Can I apply easily?\n')
    driver,note = check_easy_apply(link)

    match note:
            case 1:
                print('Yes !! Easy apply found\n')
                print('connecting ....\n')
                driver = apec_connect(driver, username, password)
                print('connected !! \n')

                if not already_applied(driver):
                    fill_and_submit(driver, cv_path, message)
                    time.sleep(2)
                    driver.quit()
                    print("Submitted, everything fine")
                    submitted_counter += 1

                else:
                    driver.quit()
                    print('Great !! I already applied to this job.')                    
                    applied_counter += 1

            case 2:
                print('Unfortunately, External website link  :(')                
                driver.quit()
                external_links_counter += 1
                external_links_list.append(link)

            case 3:
                print('probably the announcement is no longer available')
                driver.quit()
                not_available_counter += 1

________________________________________________

Link number : 1


 First step, Can I apply easily?

Yes !! Easy apply found

connecting ....

connected !! 

Application possibility.
Submitted, everything fine
________________________________________________

Link number : 2


 First step, Can I apply easily?

Yes !! Easy apply found

connecting ....

connected !! 

Application possibility.
Submitted, everything fine
________________________________________________

Link number : 3


 First step, Can I apply easily?



KeyboardInterrupt: 

#### Results

In [12]:
# Summary of the results
print('\n\nSummary of Application Process:')
print('--------------------------------')
print(f'Total links processed: {links_counter}')
print(f'Successfully submitted applications: {submitted_counter}')
print(f'Already applied: {applied_counter}')
print(f'External website links: {external_links_counter}')
print(f'Announcements no longer available: {not_available_counter}')



Summary of Application Process:
--------------------------------
Total links processed: 3
Successfully submitted applications: 2
Already applied: 0
External website links: 0
Announcements no longer available: 0
